# Revised case normalization for Hirslanden Beau-Site 2017

Notes:

- There is no data for the hospital 'Hirslanden Beau Site' in 2017 in the DB.


In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:
file_info = FileInfo(os.path.join(ROOT_DIR, 'raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx'),
                     'Hirslanden Beau Site', '2018', 'Änderungen Beau Site_ 2018')
print(file_info)


FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx', hospital_name_db='Hirslanden Beau Site', year='2018', sheet='Änderungen Beau Site_ 2018')


In [3]:
revised_cases_df = normalize(file_info)
revised_cases_df

2022-11-10 16:26:27.380 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:35 - Read 26 cases for Hirslanden Beau Site 2018
2022-11-10 16:26:27.384 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:65 - TYPES:
datum                             string
tranche                           string
case_id                           object
patient_id                        object
gender                            string
age_years                          int64
bfs_code                          string
duration_of_stay                   int64
pflegetage neu                    string
pccl                               int64
pccl neu                          string
old_pd                            string
primary_diagnosis                 string
added_icds                        string
removed_icds                      string
added_chops                       string
removed_chops                     string
drg                         

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
0,0041635636,0005050208,W,83,5,41635636,J4419,J4419,M100,3,E65C,[B965],[],[],[]
1,0041647892,0005087263,W,74,11,41647892,I350,I350,M200,3,F03D,[I490],[],[9962],[]
2,0041656218,0022162393,M,77,11,41656218,I350,I350,M200,3,F03D,"[J61, D689]",[],[],[]
3,0041660312,0005287267,W,54,30,41660312,K5722,K5722,M200,3,G02C,[E43],[E46],[],[]
4,0041670261,0005121840,M,90,6,41670261,I480,I480,M100,3,F71B,[N183],[N184],[],[]
5,0041622665,0022168673,M,79,12,41622665,C20,C20,M200,3,G17Z,[N133],[],[],[]
6,0041631826,0022164652,M,59,11,41631826,I208,I208,M200,3,F06D,"[D62, I490]",[D508],[9962],[]
7,0041632678,0005001742,W,58,5,41632678,K316,K316,M200,3,G18B,[E6692],[E6691],[],[]
8,0041642176,0001033482,M,82,10,41642176,I7024,I7024,M100,3,F13C,"[I650, I652]",[],[],[]
9,0041670429,0011088007,W,73,4,41670429,I214,I214,M100,2,F60B,"[I5001, N183, I452, I361]",[],[],[]


# Match revised DtoD data with the BfS data from the database

In [ ]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases

In [ ]:
if unmatched.shape[0] > 0:
    unmatched

In [ ]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

In [ ]:
revisions_update

In [ ]:
update_db(revisions_update, diagnoses_update, procedures_update)